<a href="https://colab.research.google.com/github/ahyz0569/STS/blob/yunok/doc/notebook/detectron2_train_ver3_customloader_200425.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# install detectron2

In [1]:
# Or, to install it from a local clone:
!git clone https://github.com/facebookresearch/detectron2.git

Cloning into 'detectron2'...
remote: Enumerating objects: 4249, done.
remote: Total 4249 (delta 0), reused 0 (delta 0), pack-reused 4249
Receiving objects: 100% (4249/4249), 2.28 MiB | 5.95 MiB/s, done.
Resolving deltas: 100% (2989/2989), done.


In [2]:
cd detectron2

/content/detectron2


In [3]:
!python -m pip install -e .

Obtaining file:///content/detectron2
  Created wheel for fvcore: filename=fvcore-0.1.dev200424-cp36-none-any.whl size=40523 sha256=5a2b738218032f4180f8db0e76fb4c094e8c6e6870550192589a235b39877fd7
  Stored in directory: /root/.cache/pip/wheels/68/20/6d/36730421a68849a49c4c7470411f9192b7fa5ec9093cf1c7b8
Successfully built fvcore
ERROR: fvcore 0.1.dev200424 has requirement pyyaml>=5.1, but you'll have pyyaml 3.13 which is incompatible.
  Running setup.py develop for detectron2


In [4]:
# name 'unicode' is not defined error 해결
!pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'

  Cloning https://github.com/cocodataset/cocoapi.git to /tmp/pip-req-build-7xo3um7s
  Running command git clone -q https://github.com/cocodataset/cocoapi.git /tmp/pip-req-build-7xo3um7s
  Created wheel for pycocotools: filename=pycocotools-2.0-cp36-cp36m-linux_x86_64.whl size=275303 sha256=7bf6b4e3e82cc9f39eabd172950de2173e18600f50cda80ab6b3e1755e6f56dc
  Stored in directory: /tmp/pip-ephem-wheel-cache-3dht3qlw/wheels/90/51/41/646daf401c3bc408ff10de34ec76587a9b3ebfac8d21ca5c3a
Successfully built pycocotools
  Found existing installation: pycocotools 2.0.0
    Uninstalling pycocotools-2.0.0:
      Successfully uninstalled pycocotools-2.0.0


In [0]:
# install detectron2:
# !pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/index.html

# library import

In [0]:
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import cv2
import random
from google.colab.patches import cv2_imshow
import os
import copy

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog

# Prepare dataset

In [0]:
from detectron2.data.datasets import register_coco_instances
register_coco_instances("my_dataset", {}, "/content/drive/My Drive/Final-project/detectron2/train/output_train.json", "/content/drive/My Drive/Final-project/detectron2/train/train_images")

In [8]:
ingre_metadata = MetadataCatalog.get("my_dataset")
ingre_metadata

Metadata(evaluator_type='coco', image_root='/content/drive/My Drive/Final-project/detectron2/train/train_images', json_file='/content/drive/My Drive/Final-project/detectron2/train/output_train.json', name='my_dataset')

In [9]:
from detectron2.data import DatasetCatalog
dataset_dicts = DatasetCatalog.get("my_dataset")

[04/26 10:50:29 d2.data.datasets.coco]: Loading /content/drive/My Drive/Final-project/detectron2/train/output_train.json takes 1.49 seconds.
WARNING [04/26 10:50:29 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[04/26 10:50:29 d2.data.datasets.coco]: Loaded 2944 images in COCO format from /content/drive/My Drive/Final-project/detectron2/train/output_train.json


In [10]:
dataset_dicts

[{'annotations': [{'bbox': [79, 0, 433, 760],
    'bbox_mode': <BoxMode.XYWH_ABS: 1>,
    'category_id': 4,
    'iscrowd': 0}],
  'file_name': '/content/drive/My Drive/Final-project/detectron2/train/train_images/0000001.jpg',
  'height': 776,
  'image_id': 1,
  'width': 600},
 {'annotations': [{'bbox': [89, 288, 399, 148],
    'bbox_mode': <BoxMode.XYWH_ABS: 1>,
    'category_id': 2,
    'iscrowd': 0},
   {'bbox': [469, 545, 330, 161],
    'bbox_mode': <BoxMode.XYWH_ABS: 1>,
    'category_id': 2,
    'iscrowd': 0},
   {'bbox': [806, 508, 449, 110],
    'bbox_mode': <BoxMode.XYWH_ABS: 1>,
    'category_id': 2,
    'iscrowd': 0}],
  'file_name': '/content/drive/My Drive/Final-project/detectron2/train/train_images/0000002.jpg',
  'height': 952,
  'image_id': 2,
  'width': 1300},
 {'annotations': [{'bbox': [59, 150, 1088, 493],
    'bbox_mode': <BoxMode.XYWH_ABS: 1>,
    'category_id': 2,
    'iscrowd': 0}],
  'file_name': '/content/drive/My Drive/Final-project/detectron2/train/train_image

# Config

In [0]:
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
import os

cfg = get_cfg()
cfg.merge_from_file(
    # Same 
    '/content/detectron2/configs/COCO-Detection/faster_rcnn_R_50_FPN_1x.yaml'
)
cfg.DATASETS.TRAIN = ("my_dataset",)
cfg.DATASETS.TEST = ()  # no metrics implemented for this dataset
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_FPN_1x.yaml")  # initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 8
# cfg.SOLVER.BASE_LR = 0.02
# cfg.SOLVER.MAX_ITER = (
#     300
# )  # 300 iterations seems good enough, but you can certainly train longer
# cfg.SOLVER.STEPS = (70000, 90000)
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = (
    128
)  # faster, and good enough for this toy dataset
# cfg.MODEL.ROI_HEADS.NUM_CLASSES = 3  # 3 classes (data, fig, hazelnut)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 8

cfg.SOLVER.CHECKPOINT_PERIOD = 2000
cfg.OUTPUT_DIR = "/content/drive/My Drive/Final-project/detectron2"

# Prepare custom dataloader

In [0]:
# from detectron2.data import build_detection_train_loader
# from detectron2.data import transforms as T
# from detectron2.data import detection_utils as utils

# def custom_mapper(dataset_dict):
#     # Implement a mapper, similar to the default DatasetMapper, but with your own customizations
#     dataset_dict = copy.deepcopy(dataset_dict)  # it will be modified by code below
#     print("dataset_dict: ", dataset_dict)
    
#     image = utils.read_image(dataset_dict["file_name"], format="BGR")
#     transform_list = [T.Resize((800, 800)),
#                       T.RandomFlip(prob=0.6, horizontal=True, vertical=False),
#                       T.RandomFlip(prob=0.6, horizontal=False, vertical=True),
#                       # T.RandomRotation(angle=[10.0, 160.0], sample_style='range'),
#                       # T.RandomContrast(0.7, 3.2),
#                       T.RandomBrightness(0.6, 1.8)
#                       ]
#     image, transforms = T.apply_transform_gens(transform_list, image)
#     dataset_dict["image"] = torch.as_tensor(image.transpose(2, 0, 1).astype("float32"))

#     annos = [
#         utils.transform_instance_annotations(obj, transforms, image.shape[:2])
#         for obj in dataset_dict.pop("annotations")
#         if obj.get("iscrowd", 0) == 0
#     ]
#     instances = utils.annotations_to_instances(annos, image.shape[:2])
#     dataset_dict["instances"] = utils.filter_empty_instances(instances)
#     return dataset_dict

# data_loader = build_detection_train_loader(cfg, mapper=custom_mapper)
# # use this dataloader instead of the default

In [0]:
# class RandomRotation2(TransformGen):
#     """
#     This method returns a copy of this image, rotated the given
#     number of degrees counter clockwise around the given center.
#     """

#     def __init__(self, angle, expand=True, center=None, sample_style="range", interp=None):
#         """
#         Args:
#             angle (list[float]): If ``sample_style=="range"``,
#                 a [min, max] interval from which to sample the angle (in degrees).
#                 If ``sample_style=="choice"``, a list of angles to sample from
#             expand (bool): choose if the image should be resized to fit the whole
#                 rotated image (default), or simply cropped
#             center (list[[float, float]]):  If ``sample_style=="range"``,
#                 a [[minx, miny], [maxx, maxy]] relative interval from which to sample the center,
#                 [0, 0] being the top left of the image and [1, 1] the bottom right.
#                 If ``sample_style=="choice"``, a list of centers to sample from
#                 Default: None, which means that the center of rotation is the center of the image
#                 center has no effect if expand=True because it only affects shifting
#         """
#         super().__init__()
#         assert sample_style in ["range", "choice"], sample_style
#         self.is_range = sample_style == "range"
#         if isinstance(angle, (float, int)):
#             angle = (angle, angle)
#         if center is not None and isinstance(center[0], (float, int)):
#             center = (center, center)
#         self._init(locals())

#     def get_transform(self, img):
#         h, w = img.shape[:2]
#         center = None
#         if self.is_range:
#             angle = np.random.uniform(self.angle[0], self.angle[1])
#             if self.center is not None:
#                 center = (
#                     np.random.uniform(self.center[0][0], self.center[1][0]),
#                     np.random.uniform(self.center[0][1], self.center[1][1]),
#                 )
#         else:
#             angle = np.random.choice(self.angle)
#             if self.center is not None:
#                 center = np.random.choice(self.center)

#         if center is not None:
#             center = (w * center[0], h * center[1])  # Convert to absolute coordinates

#         return RotationTransform(h, w, angle, expand=self.expand, center=center, interp=self.interp)

In [0]:
import copy
import logging
import numpy as np
import torch
from fvcore.common.file_io import PathManager
from PIL import Image

from detectron2.data import transforms as T
from detectron2.data import detection_utils as utils

"""
This file contains the default mapping that's applied to "dataset dicts".
"""

__all__ = ["DatasetMapper"]


class Custom_DatasetMapper:
    """
    A callable which takes a dataset dict in Detectron2 Dataset format,
    and map it into a format used by the model.
    This is the default callable to be used to map your dataset dict into training data.
    You may need to follow it to implement your own one for customized logic,
    such as a different way to read or transform images.
    See :doc:`/tutorials/data_loading` for details.
    The callable currently does the following:
    1. Read the image from "file_name"
    2. Applies cropping/geometric transforms to the image and annotations
    3. Prepare data and annotations to Tensor and :class:`Instances`
    """

    def __init__(self, cfg, is_train=True):
        if cfg.INPUT.CROP.ENABLED and is_train:
            self.crop_gen = T.RandomCrop(cfg.INPUT.CROP.TYPE, cfg.INPUT.CROP.SIZE)
            logging.getLogger(__name__).info("CropGen used in training: " + str(self.crop_gen))
        else:
            self.crop_gen = None

        self.tfm_gens = utils.build_transform_gen(cfg, is_train)

        # fmt: off
        self.img_format     = cfg.INPUT.FORMAT
        self.mask_on        = cfg.MODEL.MASK_ON
        self.mask_format    = cfg.INPUT.MASK_FORMAT
        self.keypoint_on    = cfg.MODEL.KEYPOINT_ON
        self.load_proposals = cfg.MODEL.LOAD_PROPOSALS
        # fmt: on
        if self.keypoint_on and is_train:
            # Flip only makes sense in training
            self.keypoint_hflip_indices = utils.create_keypoint_hflip_indices(cfg.DATASETS.TRAIN)
        else:
            self.keypoint_hflip_indices = None

        if self.load_proposals:
            self.min_box_side_len = cfg.MODEL.PROPOSAL_GENERATOR.MIN_SIZE
            self.proposal_topk = (
                cfg.DATASETS.PRECOMPUTED_PROPOSAL_TOPK_TRAIN
                if is_train
                else cfg.DATASETS.PRECOMPUTED_PROPOSAL_TOPK_TEST
            )
        self.is_train = is_train

    def __call__(self, dataset_dict):
        """
        Args:
            dataset_dict (dict): Metadata of one image, in Detectron2 Dataset format.
        Returns:
            dict: a format that builtin models in detectron2 accept
        """
        dataset_dict = copy.deepcopy(dataset_dict)  # it will be modified by code below
        image = utils.read_image(dataset_dict["file_name"], format="BGR")
        transform_list = [T.Resize((800, 800)),
                          T.ResizeShortestEdge(short_edge_length=(640, 672, 704, 736, 768, 800), max_size=1333, sample_style='choice'),
                        T.RandomFlip(prob=0.6, horizontal=True, vertical=False),
                        T.RandomFlip(prob=0.6, horizontal=False, vertical=True),
                        # T.RandomCrop(crop_type="relative_range", crop_size=(400, 400)),
                        T.RandomRotation(angle=[10.0, 160.0]),
                        # T.RandomContrast(0.7, 3.2),
                        T.RandomBrightness(0.9, 1.1)
                        ]
        image, transforms = T.apply_transform_gens(transform_list, image)

        # image aug check
        # flag = cv2.imwrite("/content/drive/My Drive/aa.jpg", image)
        # print(flag)
        
        dataset_dict["image"] = torch.as_tensor(image.transpose(2, 0, 1).astype("float32"))
        
        annos = [
            utils.transform_instance_annotations(obj, transforms, image.shape[:2])
            for obj in dataset_dict.pop("annotations")
            if obj.get("iscrowd", 0) == 0
        ]
        instances = utils.annotations_to_instances(annos, image.shape[:2])
        dataset_dict["instances"] = utils.filter_empty_instances(instances)
        return dataset_dict

# Train model

In [0]:
from detectron2.data import build_detection_train_loader

In [0]:
from detectron2.engine import DefaultTrainer

class Custom_Trainer(DefaultTrainer):

    @classmethod
    def build_test_loader(cls, cfg, dataset_name):
        return build_detection_test_loader(cfg, dataset_name, mapper=DatasetMapper(cfg, False))

    @classmethod
    def build_train_loader(cls, cfg):
        return build_detection_train_loader(cfg, mapper=Custom_DatasetMapper(cfg, True))

In [0]:
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
# trainer = DefaultTrainer(cfg)
trainer = Custom_Trainer(cfg)
trainer.resume_or_load(resume=True)
trainer.train()

[04/26 10:50:39 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

In [0]:
# Look at training curves in tensorboard:
%load_ext tensorboard
%tensorboard --logdir output

In [0]:
cfg

# Inference & evaluation using the trained model

In [0]:
PRETRAIN_MODEL_PATH = '/content/drive/My Drive/Final-project/detectron2'

cfg.MODEL.WEIGHTS = os.path.join(PRETRAIN_MODEL_PATH, "model_0023999.pth")
# cfg.MODEL.WEIGHTS = os.path.join('/content/output/model_final.pth')
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5   # set the testing threshold for this model
cfg.DATASETS.TEST = ("my_dataset_val2", )
predictor = DefaultPredictor(cfg)

In [0]:
from detectron2.data.datasets import register_coco_instances
register_coco_instances("my_dataset_val2", {}, "/content/drive/My Drive/Final-project/detectron2/val/output_val.json", "/content/drive/My Drive/Final-project/detectron2/val/validation_images")

In [0]:
ingre_val_metadata = MetadataCatalog.get("my_dataset_val2")
from detectron2.data import DatasetCatalog
dataset_val_dicts = DatasetCatalog.get("my_dataset_val2")

In [0]:
dataset_val_dicts

In [0]:
from detectron2.utils.visualizer import ColorMode

# for d in random.sample(dataset_val_dicts, 3):
for d in dataset_val_dicts:
    im = cv2.imread(d["file_name"])
    outputs = predictor(im)
    v = Visualizer(im[:, :, ::-1],
                   metadata=ingre_val_metadata, 
                   scale=0.8
    )
    print(outputs)
    v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    cv2_imshow(v.get_image()[:, :, ::-1])

In [0]:
im = cv2.imread('/content/drive/My Drive/Final-project/detectron2/test_image_03.jpg')
outputs = predictor(im)
v = Visualizer(im[:, :, ::-1],
                metadata=ingre_val_metadata, 
                scale=0.8
)
print(outputs)
v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
cv2_imshow(v.get_image()[:, :, ::-1])

## train image check

In [0]:
# i = 0
# for d in dataset_dicts:
#     ## onion 이미지 확인(우선 100개만)
#     if d['annotations'][0]['category_id'] == 2:
#         img = cv2.imread(d["file_name"])
#         visualizer = Visualizer(img[:, :, ::-1], metadata=ingre_metadata, scale=0.5)
#         vis = visualizer.draw_dataset_dict(d)
#         cv2_imshow(vis.get_image()[:, :, ::-1])
#         i+=1
#     if i == 10:
#         break

## evaluate its performance using AP metric implemented in COCO API

In [0]:
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
evaluator = COCOEvaluator("my_dataset_val2", cfg, False, output_dir=cfg.OUTPUT_DIR)
val_loader = build_detection_test_loader(cfg, "my_dataset_val2")
inference_on_dataset(trainer.model, val_loader, evaluator)
# another equivalent way is to use trainer.test